In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV, LinearRegression, Lasso, LassoLarsCV
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.svm import LinearSVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsRegressor, RadiusNeighborsRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

import numpy as np

import sys
sys.path.append('../src/')
from ffcDataCleaning.CodeBook import getCodeDescription

from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
gen_features = pd.read_csv('../output/imputing/.data/imputed_nodrops-v2.csv', index_col=0)
gen_features.head()

In [ ]:
specific_features = pd.read_csv('../output/imputing/.data/df_grit.csv', index_col=0)
specific_features['total'] = specific_features.sum(axis=1)
specific_features.head()
print specific_features.shape
print specific_features.columns

In [ ]:
mask = gen_features.columns[gen_features.columns.map(lambda x: x.startswith('k5')).tolist()]
child_features = gen_features[mask]

In [ ]:
features = pd.concat([gen_features,specific_features,child_features], axis = 1)
print features.shape

In [ ]:
data_dir = '../.data'
fp_train = '{}/train.csv'.format(data_dir)
df_test = pd.read_csv(fp_train,index_col=0)

In [ ]:
Y = df_test.grit.dropna()
print Y.shape
print Y.mean()
idx = Y.index

S = StandardScaler()
idx_features = features.index
FEATURES_TRAIN = S.fit_transform(features)
X = FEATURES_TRAIN[idx-1,:]
print X.shape

In [ ]:
b = 500
reg = LassoCV(cv=3, n_jobs=-1, selection='random')
#reg = GaussianProcessRegressor()
#reg = LinearRegression()
#reg = RandomForestRegressor()
#reg = MultinomialNB()
#reg = KNeighborsRegressor()
reg.fit(X[:b],Y[:b])

In [ ]:
from sklearn.feature_selection import SelectFromModel

ms = SelectFromModel(reg, threshold = .00001, prefit = True)
x_red = ms.transform(X)
impt_features = features.columns[ms.get_support()]

In [ ]:
print '\n'.join(['{:.<15s}{}'.format(x,getCodeDescription(x)) for x in impt_features])

In [ ]:
Y_pred = reg.predict(X[b:])
print mean_squared_error(Y[b:], Y_pred)

In [ ]:
plt.hist(Y_pred, normed=True)
plt.hist(Y, normed=True)

In [ ]:
reg2 = LassoCV()
reg2.fit(x_red,Y)
Y_pred = reg2.predict(x_red[b:])
print mean_squared_error(Y[b:], Y_pred)
#print reg2.alpha_

#### Best Score: 0.207
#### Baseline Score: 0.220

In [ ]:
scores = cross_val_score(LassoCV(selection='random'),x_red,Y,cv=10,scoring='neg_mean_squared_error')#,n_jobs=-1)
print abs(scores)
print scores.mean()

## Final Fit

In [ ]:
final_reg = LassoCV(selection='random', cv=10, n_jobs=-1)
final_reg.fit(x_red,Y)
full_x_red = ms.transform(FEATURES_TRAIN)
print x_red.shape
print full_x_red.shape
Y_Final = final_reg.predict(full_x_red)

In [ ]:
Y_Final = pd.DataFrame(Y_Final, index = idx_features, columns=['grit'])

In [ ]:
#sanity checks
print Y_Final.shape
print mean_squared_error(Y.values,Y_Final.loc[idx].values)
Y_Final.hist()
Y.hist()

In [ ]:
Y_Final.to_csv('../output/solutions/Solution1/grit.csv')